In [25]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [26]:
first_url = 'https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&ref_=adv_prv'

In [27]:
page = requests.get(first_url)
soup = BeautifulSoup(page.content, "html.parser")
scraped_movies = soup.find_all('div', class_='lister-item')
scraped_movies[0]

<div class="lister-item mode-advanced">
<div class="lister-top-right">
<div class="ribbonize" data-caller="filmosearch" data-tconst="tt0111161"></div>
</div>
<div class="lister-item-image float-left">
<a href="/title/tt0111161/"> <img alt="The Shawshank Redemption" class="loadlate" data-tconst="tt0111161" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BNDE3ODcxYzMtY2YzZC00NmNlLWJiNDMtZDViZWM2MzIxZDYwXkEyXkFqcGdeQXVyNjAwNDUxODI@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/S/sash/4FyxwxECzL-U1J8.png" width="67"/>
</a> </div>
<div class="lister-item-content">
<h3 class="lister-item-header">
<span class="lister-item-index unbold text-primary">1.</span>
<a href="/title/tt0111161/">The Shawshank Redemption</a>
<span class="lister-item-year text-muted unbold">(1994)</span>
</h3>
<p class="text-muted">
<span class="certificate">13+</span>
<span class="ghost">|</span>
<span class="runtime">142 min</span>
<span class="ghost">|</span>
<span class="genre">
Dr

In [28]:
def get_data(scraped_movies):
    ls = []

    for movie in scraped_movies:
        try:
            img_url = movie.find_all('div', class_='lister-item-image')[0].find_all('img', src=True)[0]['loadlate']
            title =  movie.find_all('div', class_='lister-item-image')[0].find_all('img', src=True)[0]['alt']
            year = movie.find_all('span', class_='lister-item-year')[0].get_text()
            try:
                certificate = movie.find_all('span', class_='certificate')[0].get_text()
            except:
                certificate = None
            duration = movie.find_all('span', class_='runtime')[0].get_text()
            genre = movie.find_all('span', class_='genre')[0].get_text()
            imdb = movie.find_all('div', class_='ratings-bar')[0].find_all('strong')[0].get_text()
            try:
                metascore = movie.find_all('span', class_='metascore')[0].get_text()
            except:
                metascore = None
            overview = movie.find_all('p', class_='text-muted')[1].get_text()
            director = movie.find_all('p', class_='')[0].find_all('a')[0].get_text()
            actors = []
            k = 0
            for actor in movie.find_all('p', class_='')[0].find_all('a'):
                if k != 0:
                    actors.append(actor.get_text())
                k += 1
            actors = ",".join(actors)

            tmp_ls = [img_url, title, year, certificate, duration, genre, imdb, metascore, overview, director, actors]
            ls.append(tmp_ls)
        except:
            print(title)

    return ls

In [29]:
i = 0
ls = []
while True:
    if i == 0:
        page = requests.get(first_url)
        soup = BeautifulSoup(page.content, "html.parser")
        scraped_movies = soup.find_all('div', class_='lister-item')
        ls = get_data(scraped_movies)
    else:
        url = f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start={i+1}&ref_=adv_nxt"
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        scraped_movies = soup.find_all('div', class_='lister-item')
        tmp_ls = get_data(scraped_movies)
        ls = ls + tmp_ls
    i += 50
    if i%1000==0:
        print(i)
    if i == 10000:
        print("finished")
        break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
finished


In [30]:
df = pd.DataFrame(ls, columns=["img_url", "title", "year", "certificate", "duration", "genre", "imdb", "metascore", "overview", "director", "actors"])

In [31]:
df.isna().sum()

img_url           0
title             0
year              0
certificate    3013
duration          0
genre             0
imdb              0
metascore      2021
overview          0
director          0
actors            0
dtype: int64

In [32]:
df.head(2)

,img_url,title,year,certificate,duration,genre,imdb,metascore,overview,director,actors
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,The Shawshank Redemption,(1994),13+,142 min,\nDrama,9.3,82,"\nOver the course of several years, two convic...",Frank Darabont,"Tim Robbins,Morgan Freeman,Bob Gunton,William ..."
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,(1972),18+,175 min,"\nCrime, Drama",9.2,100,"\nDon Vito Corleone, head of a mafia family, d...",Francis Ford Coppola,"Marlon Brando,Al Pacino,James Caan,Diane Keaton"


In [33]:
df['year'] = df['year'].apply(lambda x : x[1:5])

In [34]:
df['certificate'].unique()

array(['13+', '18+', 'PG', '16+', '15+', None, '7+', '6+', '13A',
       'Not Rated', 'G', 'R', '15A', 'PG-13', '10+', '7A', 'M/PG', '10A',
       'DİKKAT', 'M', '6A', 'NC-17', 'GP', 'Approved', 'Unrated',
       'YETİŞKİN', '7+13A', '14+'], dtype=object)

In [35]:
df['genre'] = df['genre'].apply(lambda x : x.split('\n')[1])

In [36]:
df['overview'] = df['overview'].apply(lambda x : x.split('\n')[1])

In [37]:
df['actors'] = df['actors'].apply(lambda x : "\t".join((x.split(','))))

In [38]:
df.to_csv('data.csv', index=False)

In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   img_url      10000 non-null  object
 1   title        10000 non-null  object
 2   year         10000 non-null  object
 3   certificate  6987 non-null   object
 4   duration     10000 non-null  object
 5   genre        10000 non-null  object
 6   imdb         10000 non-null  object
 7   metascore    7979 non-null   object
 8   overview     10000 non-null  object
 9   director     10000 non-null  object
 10  actors       10000 non-null  object
dtypes: object(11)
memory usage: 859.5+ KB


In [40]:
df.head(5)

,img_url,title,year,certificate,duration,genre,imdb,metascore,overview,director,actors
0,https://m.media-amazon.com/images/M/MV5BNDE3OD...,The Shawshank Redemption,1994,13+,142 min,Drama,9.3,82,"Over the course of several years, two convicts...",Frank Darabont,Tim Robbins\tMorgan Freeman\tBob Gunton\tWilli...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,18+,175 min,"Crime, Drama",9.2,100,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,Marlon Brando\tAl Pacino\tJames Caan\tDiane Ke...
2,https://m.media-amazon.com/images/M/MV5BNTJjZT...,Ramayana: The Legend of Prince Rama,1993,PG,135 min,"Animation, Action, Adventure",9.2,None,An anime adaptation of the Hindu epic the Rama...,Ram Mohan,Yûgô Sakô\tKoichi Saski\tArun Govil\tNikhil Ka...
3,https://m.media-amazon.com/images/M/MV5BMGRjOG...,Hababam Sinifi,1975,18+,87 min,"Comedy, Drama",9.2,None,"Lazy, uneducated students share a very close b...",Ertem Egilmez,Kemal Sunal\tMünir Özkul\tHalit Akçatepe\tTari...
4,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,16+,152 min,"Action, Crime, Drama",9.0,84,When the menace known as the Joker wreaks havo...,Christopher Nolan,Christian Bale\tHeath Ledger\tAaron Eckhart\tM...
